In [1]:
import os
import sys
import json
import numpy as np

IGRF_VERSION = 13  # should be updated with each IGRF version

CURRENT_DIR = os.path.dirname(os.path.realpath(os.getcwd()))
PARENT_DIR = os.path.dirname(os.path.dirname(CURRENT_DIR))
DATA_DIR = os.path.join(PARENT_DIR, "data")
COF_PATH = os.path.join(DATA_DIR, "IGRF{0}.COF".format(IGRF_VERSION))
JSON_PATH = os.path.join(DATA_DIR, "igrf{0}.json".format(IGRF_VERSION))

In [19]:
# initialize arrays to store content
model_arr = []
epoch_arr = []
nmain_arr = []  # max1 in original
nsv_arr = []   # max2 in original
nac_arr = []  # max2 in original
yrmin_arr = []  
yrmax_arr = []
altmin_arr = []
altmax_arr = []

In [20]:
# read the file
with open(COF_PATH, "r") as f:
    lines = [line.rstrip() for line in f]  # get each line as a string
    for line in lines:
        line_elems = str.split(line) # get contents of each line
        if len(line_elems) != 8:  # model line
#             print(line_elems)
            # truncate
            line_elems = line_elems[:-2]
#             print(line_elems)
            # unpack
            (model, epoch, nmain, nsv, nac, yrmin, yrmax, altmin, altmax) = line_elems
            
            # append to array
            model_arr.append(model)
            epoch_arr.append(epoch + "0000")  # required for reading files in C++
            nmain_arr.append(int(nmain))
            nsv_arr.append(int(nsv))
            nac_arr.append(int(nac))
            yrmin_arr.append(float(yrmin))
            yrmax_arr.append(float(yrmax))
            altmin_arr.append(float(altmin))
            altmax_arr.append(float(altmax))
            
        else:  # coefficient line
            continue  # pass for now, do this for each model

In [21]:
# check if we did this eright
print(epoch_arr)
print(type(epoch_arr[0]))
print(yrmin_arr)

['1900.000000', '1905.000000', '1910.000000', '1915.000000', '1920.000000', '1925.000000', '1930.000000', '1935.000000', '1940.000000', '1945.000000', '1950.000000', '1955.000000', '1960.000000', '1965.000000', '1970.000000', '1975.000000', '1980.000000', '1985.000000', '1990.000000', '1995.000000', '2000.000000', '2005.000000', '2010.000000', '2015.000000', '2020.000000']
<class 'str'>
[1900.0, 1905.0, 1910.0, 1915.0, 1920.0, 1925.0, 1930.0, 1935.0, 1940.0, 1945.0, 1950.0, 1955.0, 1960.0, 1965.0, 1970.0, 1975.0, 1980.0, 1985.0, 1990.0, 1995.0, 2000.0, 2005.0, 2010.0, 2015.0, 2020.0]


In [22]:
# create the framework for the json file
igrf_dict = dict((epoch, {}) for epoch in epoch_arr)
# print(igrf_dict)
for i, date in enumerate(list(igrf_dict.keys())):
    igrf_dict[date]["model"] = model_arr[i]
    igrf_dict[date]["nmain"] = nmain_arr[i]
    igrf_dict[date]["nsv"] = nsv_arr[i]
    igrf_dict[date]["nac"] = nac_arr[i]
    igrf_dict[date]["yrmin"] = yrmin_arr[i]
    igrf_dict[date]["yrmax"] = yrmax_arr[i]
    igrf_dict[date]["altmin"] = yrmin_arr[i]
    igrf_dict[date]["altmax"] = altmax_arr[i]
    
    igrf_dict[date]["gh"] = []
    igrf_dict[date]["gh_sv"] = []
# print(igrf_dict)

In [23]:
# create way to find specific date based on model
model_dict = dict((model_arr[i],epoch_arr[i]) for i in range(len(model_arr)))

In [24]:
print(igrf_dict.keys())

dict_keys(['1900.000000', '1905.000000', '1910.000000', '1915.000000', '1920.000000', '1925.000000', '1930.000000', '1935.000000', '1940.000000', '1945.000000', '1950.000000', '1955.000000', '1960.000000', '1965.000000', '1970.000000', '1975.000000', '1980.000000', '1985.000000', '1990.000000', '1995.000000', '2000.000000', '2005.000000', '2010.000000', '2015.000000', '2020.000000'])


In [25]:
# reopen the file and store the coefficients

# they are evaluated in C++ where we have something like:
# gh = [g10, h10, g11, h11, g20, h20, g21, h21, ...]
# but hn0 = 0, so these are effectively removed, resulting in:
# gh = [g10, g11, h11, g20, g21, h21, ...]

# same thing applies to secular variation coefficients

with open(COF_PATH, "r") as f:
    lines = [line.rstrip() for line in f]  # get each line as a string
    for line in lines:
        line_elems = str.split(line) # get contents of each line
        
        if len(line_elems) == 8:  # coefficient line
            # contents are given in the following order:
            # n, m, g, h, g_dot, h_dot, model, line_num

            (n, m, g, h, gdot, hdot, model, line_num) = line_elems
            
            # cast values to int / float
            n = int(n)
            m = int(m)
            g = float(g)
            h = float(h)
            gdot = float(gdot)
            hdot = float(hdot)

            # find date specific to each model
            date = model_dict[model]
            
            # append coefficients and secular variation
            igrf_dict[date]["gh"].append(g)
            igrf_dict[date]["gh_sv"].append(gdot)
            if m != 0:  # only append h when m != 0
                igrf_dict[date]["gh"].append(h)
                igrf_dict[date]["gh_sv"].append(hdot)

In [27]:
# check if we did this correctly
print(igrf_dict["2020.000000"]["gh_sv"])

[5.7, 7.4, -25.9, -11.0, -7.0, -30.2, -2.1, -22.4, 2.2, -5.9, 6.0, 3.1, -1.1, -12.0, 0.5, -1.2, -1.6, -0.1, -5.9, 6.5, 5.2, 3.6, -5.1, -5.0, -0.3, 0.5, 0.0, -0.6, 2.5, 0.2, -0.6, 1.3, 3.0, 0.9, 0.3, -0.5, -0.3, 0.0, 0.4, -1.6, 1.3, -1.3, -1.4, 0.8, 0.0, 0.0, 0.9, 1.0, -0.1, -0.2, 0.6, 0.0, 0.6, 0.7, -0.8, 0.1, -0.2, -0.5, -1.1, -0.8, 0.1, 0.8, 0.3, 0.0, 0.1, -0.2, -0.1, 0.6, 0.4, -0.2, -0.1, 0.5, 0.4, -0.3, 0.3, -0.4, -0.1, 0.5, 0.4, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [5]:
# check here if json file is properly unpacked

with open(JSON_PATH, "r") as f:
    j = json.load(f)

print(j["1900.000000"]["gh"])

[-31543.0, -2298.0, 5922.0, -677.0, 2905.0, -1061.0, 924.0, 1121.0, 1022.0, -1469.0, -330.0, 1256.0, 3.0, 572.0, 523.0, 876.0, 628.0, 195.0, 660.0, -69.0, -361.0, -210.0, 134.0, -75.0, -184.0, 328.0, -210.0, 264.0, 53.0, 5.0, -33.0, -86.0, -124.0, -16.0, 3.0, 63.0, 61.0, -9.0, -11.0, 83.0, -217.0, 2.0, -58.0, -35.0, 59.0, 36.0, -90.0, -69.0, 70.0, -55.0, -45.0, 0.0, -13.0, 34.0, -10.0, -41.0, -1.0, -21.0, 28.0, 18.0, -12.0, 6.0, -22.0, 11.0, 8.0, 8.0, -4.0, -14.0, -9.0, 7.0, 1.0, -13.0, 2.0, 5.0, -9.0, 16.0, 5.0, -5.0, 8.0, -18.0, 8.0, 10.0, -20.0, 1.0, 14.0, -11.0, 5.0, 12.0, -3.0, 1.0, -2.0, -2.0, 8.0, 2.0, 10.0, -1.0, -2.0, -1.0, 2.0, -3.0, -4.0, 2.0, 2.0, 1.0, -5.0, 2.0, -2.0, 6.0, 6.0, -4.0, 4.0, 0.0, 0.0, -2.0, 2.0, 4.0, 2.0, 0.0, 0.0, -6.0]


In [9]:
x = j["1905.000000"]["gh"]
print(len(x))

120
